# Install

In [ ]:
# !pip install camelot-py[cv] pandas openpyxl

# Import

In [ ]:
import pandas as pd
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)

# fn

In [ ]:
from modules.process_pdf import process_pdf
from modules.extract_logic import clean_row
from config.schedule import Schedule80U

In [ ]:
from typing import Any, List
def get_indexes(lst: List[Any], target: Any) -> List[int]:
    return [i for i, val in enumerate(lst) if val == target]

# Code

In [ ]:
extracted_row = [
    # ['SECTION 80U - DETAILS OF DEDUCTION IN CASE OF A PERSON WITH DISABILITY', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None],
    ['Sl.\nNo.', None, 'Nature of Disability', None, None, None, None, 'Type of Disability', None, None, 'Amount of Deduction', None, None, None, None, 'Ack. No. of Form 10IA\nfiled', None, None, None, 'UDID Number (If\navailable)', None],
    ['(1)', None, '(2a)', None, None, None, None, '(2b)', None, None, '(3)', None, None, None, None, '(4)', None, None, None, '(5)', None],
    ['1', None, '', None, None, None, None, '', None, None, '0', None, None, None, None, '', None, None, None, '', None],
]
df = pd.DataFrame(extracted_row[-1:])
# df = clean_row(df)
df

In [ ]:
# Assume df is your cleaned DataFrame
header_row = df.iloc[1]
header_map = {val: idx for idx, val in header_row.items() if pd.notna(val)}
header_map

In [ ]:
entries = []
for i in range(2, len(df)):
    row = df.iloc[i]
    entry = Schedule80U.from_dataframe_row(row, header_map)
    entries.append(entry)
entries

In [ ]:
a=['(1)', None, '(2a)', None, None, None, None, '(2b)', None, None, '(3)', None, None, None, None, '(4)', None, None, None, '(5)', None]
non_empty_indexes = [i for i, val in enumerate(a) if val not in (None, "")]
print("Indexes of non-empty values:", non_empty_indexes)
b=['1', None, '', None, None, None, None, '', None, None, '0', None, None, None, None, '', None, None, None, '', None]
# Example 3: Get indexes of all non-empty values
non_empty_indexes = [i for i, val in enumerate(b) if val not in (None, "")]
print("Indexes of non-empty values:", non_empty_indexes)

In [ ]:
input_file_path = 'OUTPUT/ ITR 1-2 AY 2025-26.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
EXTRACTED_DATA=process_pdf(input_file_path,output_file_path)

In [ ]:
assert False

In [ ]:
class schedule

In [ ]:
def parse_with_markers(extracted_data, schedule_name="SCHEDULE IT"):
    headers = None
    column_markers = None
    rows = []

    for row in extracted_data:
        if any(schedule_name in str(cell) for cell in row if cell is not None):
            capture = True
            continue
        if capture:
            if headers is None:
                headers = [c for c in row]
                continue
        if column_markers is None:
            column_markers = [c for c in row if c is not None]
            continue

        # Use column_markers length to filter row
        filtered = [c for c in row if c is not None]
        if len(filtered) == len(column_markers):
            rows.append(filtered)

    # Build DataFrame
    if headers and rows:
        # Align headers to markers (keep only marker columns)
        keep_indices = [i for i, c in enumerate(headers) if f"({i+1})" in column_markers or headers[i] in ["Sl.\nNo.", "BSR Code", "Date of Deposit (DD/MM/YYYY)", "Serial Number of Challan", "Tax paid"]]
        final_headers = [headers[i] for i in keep_indices]

        cleaned_rows = []
        for row in rows:
            # Map row values to keep_indices
            cleaned_rows.append([row[i] for i in range(len(row)) if i < len(final_headers)])

        df = pd.DataFrame(cleaned_rows, columns=final_headers)
        return df
    else:
        return pd.DataFrame()

# Run parser
df_schedule_it = parse_with_markers(EXTRACTED_DATA)
df_schedule_it